# Logistic Regression (LogReg) Example

This is a SIMPLE sample implementation of Logistic Regression using the `turtles` `LogReg` class and various supporting functions.

The `LogReg` class implements Logistic Regression using Maximum Likelihood Estimation (MLE) for parameter estimation.

In [1]:
import time

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from turtles.stats.glms import LogReg
from turtles.stats import (
    variance_inflation_factor,
    pca
)

In [2]:
random_state = 5
test_size = 0.2

In [3]:
# load sample data
X, y = load_breast_cancer(return_X_y=True)

print("X:", X.shape)
print("y:", y.shape)

X: (569, 30)
y: (569,)


In [4]:
# create splits
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, 
    y, 
    test_size=test_size, 
    random_state=random_state
)

Ytrain = Ytrain.reshape(Ytrain.shape[0], 1)
Ytest = Ytest.reshape(Ytest.shape[0], 1)

In [5]:
# EDA

# check for multicollinearity
vifs = variance_inflation_factor(Xtrain)
print("VIF Table:")
display(vifs)

VIF Table:


,Coefficient,R-squared,VIF
0,x0,0.999729,3692.842120
1,x1,0.911654,11.319176
2,x2,0.999730,3702.834171
3,x3,0.996914,324.095944
4,x4,0.878489,8.229712
5,x5,0.977671,44.784059
6,x6,0.986572,74.472502
7,x7,0.982589,57.434044
8,x8,0.763329,4.225272
9,x9,0.932192,14.747468


In [6]:
# there appears to be significant multicollinearity
# let's use PCA to reduce the dimensions, then re-split the data
X = pca(X, 2)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=5
)

Ytrain = Ytrain.reshape(Ytrain.shape[0], 1)
Ytest = Ytest.reshape(Ytest.shape[0], 1)

print("Xtrain:", Xtrain.shape)
print("Xtest:", Xtest.shape)
print("Ytrain:", Ytrain.shape)
print("Ytest", Ytest.shape)
print()

Xtrain: (455, 2)
Xtest: (114, 2)
Ytrain: (455, 1)
Ytest (114, 1)



In [7]:
# check for multicollinearity again
vifs = variance_inflation_factor(Xtrain)
print("VIF Table:")
display(vifs)

VIF Table:


,Coefficient,R-squared,VIF
0,x0,0.011005,1.011127
1,x1,0.011005,1.011127


In [8]:
# fit model using newtons method
# tune hyperparameters as needed
n_model = LogReg(
    method="newton",
    learning_rate=0.1,
    tolerance=0.00001
)

In [9]:
start = time.time()
n_model.fit(Xtrain, Ytrain)
print("Newtons time to fit:", time.time() - start)
print("Newtons Iterations:", n_model.iterations)
print()

print("Newtons Method Model Summary:\n")
display(n_model.summary())

Newtons time to fit: 0.01765751838684082
Newtons Iterations: 120

Newtons Method Model Summary:



,Variable,Coefficient,Std Error,z-statistic,p-value,[0.025,0.075]
0,Intercept,-0.8607,0.2993,-2.8759,0.004,-1.4474,-0.2741
1,x0,0.0119,0.0013,8.8113,0.000,0.0092,0.0145
2,x1,-0.0290,0.0051,-5.6576,0.000,-0.0390,-0.0189


In [10]:
# results on test data using 0.5 as threshold
n_preds = n_model.predict(Xtest) > 0.5

print("TEST Accuracy:", np.sum(n_preds == Ytest) / len(Ytest))

TEST Accuracy: 0.9473684210526315


## Compare to Statsmodels

Let's compare the `turtles` implementation to `statsmodels`.

In [11]:
import statsmodels.api as sm

In [12]:
Xtrain = sm.add_constant(Xtrain)

start = time.time()
log_reg = sm.Logit(Ytrain, Xtrain).fit()
print()
print("Statsmodels time to converage: ", time.time() - start)
print()
print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.172636
         Iterations 9

Statsmodels time to converage:  0.005274057388305664

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  455
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            2
Date:                Sat, 04 Oct 2025   Pseudo R-squ.:                  0.7359
Time:                        12:40:27   Log-Likelihood:                -78.549
converged:                       True   LL-Null:                       -297.42
Covariance Type:            nonrobust   LLR p-value:                 8.823e-96
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8608      0.299     -2.876      0.00

In [13]:
Xtest = sm.add_constant(Xtest)
sm_preds = log_reg.predict(Xtest) > 0.5
sm_preds = sm_preds.reshape((Ytest.shape[0], 1))

print("Statsmodels TEST Accuracy:", np.sum(sm_preds == Ytest) / len(Ytest))

Statsmodels TEST Accuracy: 0.9473684210526315
